# 正規方程式の実装

このノートブックでは、線形回帰における正規方程式（Normal Equation）の実装を行います。

## 目次
1. [正規方程式の理論](#正規方程式の理論)
2. [基本的な実装](#基本的な実装)
3. [数値的安定性の考慮](#数値的安定性の考慮)
4. [最急降下法との比較](#最急降下法との比較)
5. [実データでの検証](#実データでの検証)
6. [演習問題](#演習問題)


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import time

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 乱数の固定
np.random.seed(42)

print("ライブラリのインポートが完了しました。")


## 正規方程式の理論

正規方程式は、最小二乗法の解を直接的に求める手法です。

### 数学的導出

**目的関数**:
```
J(θ) = (1/2m) × ||Xθ - y||²
```

**勾配を0とおく**:
```
∇J(θ) = Xᵀ(Xθ - y) = 0
```

**正規方程式**:
```
XᵀXθ = Xᵀy
```

**解**:
```
θ = (XᵀX)⁻¹Xᵀy
```

### 利点と制約

**利点**:
- 解析解が得られる
- 反復計算が不要
- 確定的な結果

**制約**:
- 計算量がO(n³)
- メモリ使用量が多い
- 多重共線性で不安定


## 基本的な実装

正規方程式の基本的な実装を行います。


In [ ]:
# サンプルデータの生成
np.random.seed(42)

# 住宅価格予測の例
area = np.random.uniform(30, 120, 100)
price = 50 * area + np.random.normal(0, 100, 100)

# データフレームに変換
data = pd.DataFrame({
    'area': area,
    'price': price
})

# 特徴量と目的変数の準備
X = data[['area']].values
y = data['price'].values

# バイアス項の追加
X_with_bias = np.c_[np.ones(X.shape[0]), X]

print("データの基本情報:")
print(f"サンプル数: {X.shape[0]}")
print(f"特徴量数: {X.shape[1]}")
print(f"バイアス項追加後の形状: {X_with_bias.shape}")

# データの可視化
plt.figure(figsize=(10, 6))
plt.scatter(data['area'], data['price'], alpha=0.6, color='blue')
plt.xlabel('面積 (m²)')
plt.ylabel('価格 (万円)')
plt.title('住宅価格データ')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
def normal_equation(X, y):
    """
    正規方程式を解く関数
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        特徴量行列（バイアス項を含む）
    y : array-like, shape (n_samples,)
        目的変数
    
    Returns:
    theta : array-like, shape (n_features,)
        回帰係数
    """
    # 正規方程式: θ = (X^T X)^(-1) X^T y
    XtX = X.T.dot(X)
    Xty = X.T.dot(y)
    
    # 逆行列の計算
    try:
        theta = np.linalg.inv(XtX).dot(Xty)
    except np.linalg.LinAlgError:
        print("警告: 行列が特異です。擬似逆行列を使用します。")
        theta = np.linalg.pinv(XtX).dot(Xty)
    
    return theta

# 正規方程式の実行
print("正規方程式の実行:")
print("=" * 50)

start_time = time.time()
theta_ne = normal_equation(X_with_bias, y)
end_time = time.time()

print(f"計算時間: {end_time - start_time:.6f}秒")
print(f"最適化されたパラメータ:")
print(f"切片 (β₀): {theta_ne[0]:.6f}")
print(f"傾き (β₁): {theta_ne[1]:.6f}")

# 予測値の計算
y_pred = X_with_bias.dot(theta_ne)

# 評価指標の計算
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print(f"\n評価指標:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

# 結果の可視化
plt.figure(figsize=(12, 5))

# 回帰直線の描画
plt.subplot(1, 2, 1)
x_range = np.linspace(data['area'].min(), data['area'].max(), 100)
y_line = theta_ne[0] + theta_ne[1] * x_range

plt.scatter(data['area'], data['price'], alpha=0.6, color='blue', label='データ')
plt.plot(x_range, y_line, 'r-', linewidth=2, label='回帰直線')
plt.xlabel('面積 (m²)')
plt.ylabel('価格 (万円)')
plt.title('正規方程式による回帰直線')
plt.legend()
plt.grid(True, alpha=0.3)

# 残差プロット
plt.subplot(1, 2, 2)
residuals = y - y_pred
plt.scatter(y_pred, residuals, alpha=0.6, color='green')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('予測値')
plt.ylabel('残差')
plt.title('残差プロット')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 数値的安定性の考慮

正規方程式では、行列の条件数や多重共線性の問題を考慮する必要があります。


In [ ]:
def robust_normal_equation(X, y, regularization=1e-6):
    """
    数値的安定性を考慮した正規方程式
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        特徴量行列
    y : array-like, shape (n_samples,)
        目的変数
    regularization : float
        正則化パラメータ（Ridge正則化）
    
    Returns:
    theta : array-like, shape (n_features,)
        回帰係数
    """
    # 正規方程式: θ = (X^T X + λI)^(-1) X^T y
    XtX = X.T.dot(X)
    Xty = X.T.dot(y)
    
    # 条件数の確認
    condition_number = np.linalg.cond(XtX)
    print(f"行列の条件数: {condition_number:.2e}")
    
    if condition_number > 1e12:
        print("警告: 行列の条件数が大きすぎます。正則化を適用します。")
        # Ridge正則化: (X^T X + λI)^(-1) X^T y
        n_features = XtX.shape[0]
        regularized_XtX = XtX + regularization * np.eye(n_features)
        theta = np.linalg.solve(regularized_XtX, Xty)
    else:
        # 通常の逆行列計算
        try:
            theta = np.linalg.solve(XtX, Xty)
        except np.linalg.LinAlgError:
            print("警告: 行列が特異です。擬似逆行列を使用します。")
            theta = np.linalg.pinv(XtX).dot(Xty)
    
    return theta

# 数値的安定性のテスト
print("数値的安定性のテスト:")
print("=" * 50)

# 通常のデータでの実行
theta_robust = robust_normal_equation(X_with_bias, y)
print(f"通常データでの結果: {theta_robust}")

# 多重共線性のあるデータでのテスト
print("\n多重共線性のあるデータでのテスト:")
X_collinear = np.column_stack([
    np.ones(100),
    np.random.normal(0, 1, 100),
    np.random.normal(0, 1, 100) + 0.99 * np.random.normal(0, 1, 100)  # 高い相関
])

y_collinear = 2 * X_collinear[:, 1] + 3 * X_collinear[:, 2] + np.random.normal(0, 0.1, 100)

theta_collinear = robust_normal_equation(X_collinear, y_collinear)
print(f"多重共線性データでの結果: {theta_collinear}")

# 条件数の可視化
def plot_condition_number():
    """条件数の影響を可視化"""
    # 異なる相関を持つデータを生成
    correlations = [0.1, 0.5, 0.8, 0.9, 0.95, 0.99]
    condition_numbers = []
    
    for corr in correlations:
        # 相関のあるデータを生成
        x1 = np.random.normal(0, 1, 100)
        x2 = corr * x1 + np.sqrt(1 - corr**2) * np.random.normal(0, 1, 100)
        
        X_test = np.column_stack([np.ones(100), x1, x2])
        XtX = X_test.T.dot(X_test)
        condition_numbers.append(np.linalg.cond(XtX))
    
    plt.figure(figsize=(10, 6))
    plt.semilogy(correlations, condition_numbers, 'bo-', markersize=8)
    plt.axhline(y=1e12, color='r', linestyle='--', label='危険な条件数 (1e12)')
    plt.xlabel('特徴量間の相関')
    plt.ylabel('条件数 (対数スケール)')
    plt.title('相関と条件数の関係')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return correlations, condition_numbers

correlations, condition_numbers = plot_condition_number()

print(f"\n相関と条件数の関係:")
for corr, cond in zip(correlations, condition_numbers):
    print(f"相関 {corr:4.2f}: 条件数 {cond:8.2e}")


## 最急降下法との比較

正規方程式と最急降下法の性能を比較します。


In [ ]:
# 最急降下法の実装（簡略版）
def gradient_descent_simple(X, y, learning_rate=0.01, max_iterations=1000):
    """簡略版の最急降下法"""
    theta = np.zeros(X.shape[1])
    m = len(y)
    
    for i in range(max_iterations):
        predictions = X.dot(theta)
        error = predictions - y
        gradient = (1/m) * X.T.dot(error)
        theta = theta - learning_rate * gradient
        
        # 収束判定
        if np.linalg.norm(gradient) < 1e-6:
            break
    
    return theta, i + 1

# 性能比較
def compare_methods():
    """正規方程式と最急降下法の比較"""
    print("性能比較:")
    print("=" * 50)
    
    # 正規方程式
    start_time = time.time()
    theta_ne = normal_equation(X_with_bias, y)
    ne_time = time.time() - start_time
    
    # 最急降下法
    start_time = time.time()
    theta_gd, iterations = gradient_descent_simple(X_with_bias, y, learning_rate=0.01)
    gd_time = time.time() - start_time
    
    # 結果の比較
    print(f"正規方程式:")
    print(f"  計算時間: {ne_time:.6f}秒")
    print(f"  パラメータ: {theta_ne}")
    
    print(f"\n最急降下法:")
    print(f"  計算時間: {gd_time:.6f}秒")
    print(f"  イテレーション数: {iterations}")
    print(f"  パラメータ: {theta_gd}")
    
    # 精度の比較
    y_pred_ne = X_with_bias.dot(theta_ne)
    y_pred_gd = X_with_bias.dot(theta_gd)
    
    mse_ne = mean_squared_error(y, y_pred_ne)
    mse_gd = mean_squared_error(y, y_pred_gd)
    
    print(f"\n精度比較:")
    print(f"正規方程式 - MSE: {mse_ne:.6f}")
    print(f"最急降下法 - MSE: {mse_gd:.6f}")
    print(f"差: {abs(mse_ne - mse_gd):.8f}")
    
    return theta_ne, theta_gd, ne_time, gd_time

theta_ne, theta_gd, ne_time, gd_time = compare_methods()

# スケーラビリティのテスト
def scalability_test():
    """データサイズに対する性能の変化"""
    sample_sizes = [100, 500, 1000, 2000, 5000]
    ne_times = []
    gd_times = []
    
    for n_samples in sample_sizes:
        # データ生成
        X_test = np.random.normal(0, 1, (n_samples, 1))
        y_test = 2 * X_test[:, 0] + np.random.normal(0, 0.1, n_samples)
        X_test_bias = np.c_[np.ones(n_samples), X_test]
        
        # 正規方程式
        start_time = time.time()
        normal_equation(X_test_bias, y_test)
        ne_times.append(time.time() - start_time)
        
        # 最急降下法
        start_time = time.time()
        gradient_descent_simple(X_test_bias, y_test, learning_rate=0.01)
        gd_times.append(time.time() - start_time)
    
    # 結果の可視化
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(sample_sizes, ne_times, 'bo-', label='正規方程式')
    plt.plot(sample_sizes, gd_times, 'ro-', label='最急降下法')
    plt.xlabel('サンプル数')
    plt.ylabel('計算時間 (秒)')
    plt.title('スケーラビリティの比較')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.loglog(sample_sizes, ne_times, 'bo-', label='正規方程式')
    plt.loglog(sample_sizes, gd_times, 'ro-', label='最急降下法')
    plt.xlabel('サンプル数 (対数スケール)')
    plt.ylabel('計算時間 (対数スケール)')
    plt.title('スケーラビリティの比較 (対数スケール)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return sample_sizes, ne_times, gd_times

sample_sizes, ne_times, gd_times = scalability_test()

print(f"\nスケーラビリティの結果:")
print("サンプル数 | 正規方程式 | 最急降下法")
print("-" * 40)
for n, ne_time, gd_time in zip(sample_sizes, ne_times, gd_times):
    print(f"{n:8d} | {ne_time:10.6f} | {gd_time:10.6f}")


## 実データでの検証

scikit-learnとの比較を行い、実装の正確性を検証します。


In [ ]:
# scikit-learnとの比較
from sklearn.linear_model import LinearRegression

# scikit-learnの線形回帰
lr_sklearn = LinearRegression()
lr_sklearn.fit(X, y)

# 結果の比較
print("scikit-learnとの比較:")
print("=" * 50)
print(f"正規方程式 - 切片: {theta_ne[0]:.6f}, 傾き: {theta_ne[1]:.6f}")
print(f"scikit-learn - 切片: {lr_sklearn.intercept_:.6f}, 傾き: {lr_sklearn.coef_[0]:.6f}")

# 予測の比較
y_pred_ne = X_with_bias.dot(theta_ne)
y_pred_sklearn = lr_sklearn.predict(X)

# 予測精度の比較
mse_ne = mean_squared_error(y, y_pred_ne)
mse_sklearn = mean_squared_error(y, y_pred_sklearn)

print(f"\n予測精度の比較:")
print(f"正規方程式 - MSE: {mse_ne:.6f}")
print(f"scikit-learn - MSE: {mse_sklearn:.6f}")
print(f"差: {abs(mse_ne - mse_sklearn):.8f}")

# パラメータの差
param_diff = np.linalg.norm(theta_ne - np.array([lr_sklearn.intercept_, lr_sklearn.coef_[0]]))
print(f"パラメータの差: {param_diff:.8f}")

# 可視化
plt.figure(figsize=(15, 5))

# 回帰直線の比較
plt.subplot(1, 3, 1)
x_range = np.linspace(data['area'].min(), data['area'].max(), 100)
y_ne = theta_ne[0] + theta_ne[1] * x_range
y_sklearn = lr_sklearn.intercept_ + lr_sklearn.coef_[0] * x_range

plt.scatter(data['area'], data['price'], alpha=0.6, color='blue', label='データ')
plt.plot(x_range, y_ne, 'r-', linewidth=2, label='正規方程式')
plt.plot(x_range, y_sklearn, 'g--', linewidth=2, label='scikit-learn')
plt.xlabel('面積 (m²)')
plt.ylabel('価格 (万円)')
plt.title('回帰直線の比較')
plt.legend()
plt.grid(True, alpha=0.3)

# 予測値の比較
plt.subplot(1, 3, 2)
plt.scatter(y_pred_ne, y_pred_sklearn, alpha=0.6, color='purple')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', label='完全一致')
plt.xlabel('正規方程式の予測値')
plt.ylabel('scikit-learnの予測値')
plt.title('予測値の比較')
plt.legend()
plt.grid(True, alpha=0.3)

# 残差の比較
plt.subplot(1, 3, 3)
residuals_ne = y - y_pred_ne
residuals_sklearn = y - y_pred_sklearn
plt.scatter(y_pred_ne, residuals_ne, alpha=0.6, color='red', label='正規方程式')
plt.scatter(y_pred_sklearn, residuals_sklearn, alpha=0.6, color='green', label='scikit-learn')
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('予測値')
plt.ylabel('残差')
plt.title('残差の比較')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 統計的検定
from scipy import stats

# パラメータの差の検定
t_stat, p_value = stats.ttest_1samp(theta_ne - np.array([lr_sklearn.intercept_, lr_sklearn.coef_[0]]), 0)
print(f"\n統計的検定:")
print(f"t統計量: {t_stat:.6f}")
print(f"p値: {p_value:.6f}")

if p_value > 0.05:
    print("結論: パラメータに有意な差はありません (p > 0.05)")
else:
    print("結論: パラメータに有意な差があります (p ≤ 0.05)")


## 演習問題

### 問題1: 正則化の実装
Ridge正則化を正規方程式に追加し、正則化パラメータの影響を調査してください。

### 問題2: 多重共線性の対処
多重共線性のあるデータに対して、主成分分析（PCA）を適用してから正規方程式を解いてください。

### 問題3: 大規模データでの性能
大規模データセット（10,000サンプル以上）での正規方程式と最急降下法の性能を比較してください。

### 問題4: 数値的安定性の改善
条件数が大きい場合の数値的安定性を改善する手法を実装してください。


## まとめ

このノートブックでは、正規方程式の実装を行いました：

### 実装した機能
1. **基本的な正規方程式**: 解析解の直接計算
2. **数値的安定性**: 条件数と正則化の考慮
3. **性能比較**: 最急降下法との比較
4. **検証**: scikit-learnとの比較

### 重要なポイント
- **計算量**: O(n³)の計算量
- **数値的安定性**: 条件数と多重共線性の考慮
- **正則化**: Ridge正則化による安定化
- **スケーラビリティ**: 大規模データでの制約

### 使用場面の判断
- **正規方程式を選ぶべき場合**: 小規模データ、確実な解が必要
- **最急降下法を選ぶべき場合**: 大規模データ、メモリ制約がある

### 次のステップ
- [基本的な線形回帰](./linear_regression_basic.ipynb)
- [最急降下法の実装](./gradient_descent_implementation.ipynb)
- [特徴量スケーリング](../02_feature_scaling/02_feature_scaling.md)
